## Video

Images are great, but since optimization happens over time video are able to capture the optimization process itself often in a better way.
In `pymoo` we offer a wrapper around `matplotlib` to combine heterogeneous plots and and put them together in a video (a different way than the animation package does it). This is, however, more computationally expensive, but makes recording very simple.

To enable video support you have to install `pyrecorder` by

```
pip install -U pyrecorder
```

Because our recording tool has some dependencies not every regular `pymoo` user would be interest in, we have decided to outsource the recording to another third-party library.

For instance, let us record a very short video with only three frames (randomly created scatter plots):

In [1]:
import numpy as np
from pyrecorder.recorders.file import File
from pyrecorder.video import Video

from pymoo.algorithms.nsga2 import NSGA2
from pymoo.visualization.scatter import Scatter

vid = Video(File("example.mp4"))

for k in range(10):
    X = np.random.random((100, 2))
    Scatter(title=str(k)).add(X).do()
    vid.record()

vid.close()

Or recording a video after the run has finished:

In [2]:
from pymoo.factory import get_problem
from pymoo.optimize import minimize

problem = get_problem("zdt1")

algorithm = NSGA2(pop_size=100, eliminate_duplicates=True)

ret = minimize(problem,
               algorithm,
               termination=('n_gen', 100),
               seed=1,
               save_history=True,
               verbose=False)

# use the video writer as a resource
with Video(File("ga.mp4")) as vid:

    # for each algorithm object in the history
    for entry in ret.history:
        sc = Scatter(title=("Gen %s" % entry.n_gen))
        sc.add(entry.pop.get("F"))
        sc.add(entry.problem.pareto_front(), plot_type="line", color="black", alpha=0.7)
        sc.do()

        # finally record the current visualization to the video
        vid.record()

To stream the the current algorithm state directly to the screen the callback directive can be used to initiate a video by `Video(Streamer())`.
This will allow to stream the current status of the algorithm.

In [3]:
from pyrecorder.recorders.streamer import Streamer
from pyrecorder.video import Video

from pymoo.algorithms.so_genetic_algorithm import GA
from pymoo.factory import get_problem
from pymoo.model.callback import Callback
from pymoo.optimize import minimize
from pymoo.visualization.pcp import PCP


class MyCallback(Callback):

    def __init__(self) -> None:
        super().__init__()
        self.video = Video(Streamer())

    def notify(self, algorithm):
        problem = algorithm.problem

        pcp = PCP(title=("Gen %s" % algorithm.n_gen, {'pad': 30}),
                  bounds=(problem.xl, problem.xu),
                  labels=["$x_%s$" % k for k in range(problem.n_var)]
                  )
        pcp.set_axis_style(color="grey", alpha=0.5)

        pcp.add(algorithm.pop.get("X"), color="black", alpha=0.8, linewidth=1.5)
        if algorithm.off is not None:
            pcp.add(algorithm.off.get("X"), color="blue", alpha=0.8, linewidth=0.5)

        pcp.add(algorithm.opt.get("X"), color="red", linewidth=4)
        pcp.do()

        self.video.record()


problem = get_problem("rastrigin", n_var=10)

algorithm = GA(pop_size=50, eliminate_duplicates=True, callback=MyCallback())

ret = minimize(problem,
               algorithm,
               termination=('n_gen', 50),
               seed=1,
               verbose=False)
